## Import libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import expon

import pyximport;
pyximport.install()

from entropic_smcTranslocator import smcTranslocatorDirectional 

from brandaolib.contactProbability_generator import ChainLayout
from itertools import product


import pandas as pd
import os
import re
from scipy.sparse import coo_matrix

from multiprocessing import Pool
import pickle


/home/hbrandao/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/hbrandao/libs/MCMs-as-random-barriers-to-loop-extrusion-paper/entropic_smcTranslocator.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


## Define loop strength calculation

In [2]:
def getLoopStrength(Z,Zcount,loopBases,pad=16,shift=50):
    ZZ = Z/Zcount
    ZZ[np.isnan(ZZ)] = 0
    np.fill_diagonal(ZZ,np.mean(np.diag(ZZ))/2)
    data = ZZ + ZZ.T
    
    # skip first and last TADs
    loopStrengthsList = [] 
    for (stBin,endBin) in zip(loopBases[2:],loopBases[1:-1]):
        MM = data[stBin - pad:stBin + pad, endBin - pad:endBin + pad]
        MC = data[stBin - pad+shift:stBin + pad+shift, endBin - pad+shift:endBin + pad+shift]
        
        M = MM/MC
        
        M[np.isinf(M)] = 0
        # divide box into 3 equal spaces
        L = len(M)
        box1 = M[0:L//3,0:L//3] 
        box2 = M[L//2-L//6:L//2+L//6,L//2-L//6:L//2+L//6] 
        box3 = M[L-L//3:L,L-L//3:L] 
        
        loopStrengthsList.append(np.nansum(box2)/(np.nansum(box3)+np.nansum(box1))*2)
    return loopStrengthsList


## Define range of parameter sweep

In [4]:
# fixed boundaries
nreps = 1000 #10000 #20000
nsamples = 70
fractionBoundaries = [0]#[2,4,8,16]#[0,0.24,0.5,1,1.6,2]
boundaryPauseProb = [0]#[0.5,0.25,0.125,0.0625,0.03125]#[1,0.75,0.5,0.25]
processivities = [60,75,95,100]#[100,150,200,250,300]#[100,200,300]
separations = [75, 100, 150,200]#[100,150,200,250,300]#[200]

TAD_size = 300
TAD_boundaries = np.arange(0,10000,TAD_size)

name_format_string = './RandomBoundarySims_refined/FixedTADs_proc{}_sep{}_tadsize{}_fractionBoundaries{}_boundaryPauseProb{}_nsamples{}_nreps{}.pkl'
lof_nums = list(product(fractionBoundaries,boundaryPauseProb,processivities,separations))
lof = [(x[0],x[1],name_format_string.format(x[2],x[3],TAD_size,x[0],x[1],nreps,nsamples),x[2],x[3]) for x in lof_nums]


len(lof)

16

In [23]:
# # random boundaries
# nreps = 20000
# nsamples = 70
# fractionBoundaries = [4,8,16]#[0,0.24,0.5,1,1.6,2]
# boundaryPauseProb = [0.5,0.25,0.125,0.0625]#[1,0.75,0.5,0.25]

# lof_nums = list(product(fractionBoundaries,boundaryPauseProb))

# lof = [(x[0],x[1],'./RandomBoundarySims/fractionBoundaries{}_boundaryPauseProb{}_nsamples{}_nreps{}.pkl'
#                                 .format(x[0],x[1],nreps,nsamples)) for x in lof_nums]


# TAD_size = 200
# TAD_boundaries =##np.random.choice(10000,int(10000/TAD_size),replace=False)

## Run the simulation for all parameters (parallel)

In [ ]:
def doOne(idx):
    try:
        filename = lof[idx][2]

        #unchanging parameters
        v_extruder =  50 # kb/min  
        BELT_ON=0
        BELT_OFF=1
        switchRate = 0 
        SWITCH_PROB= switchRate # switching rate
        PUSH=0
        PAIRED=0
        SLIDE=1
        SLIDE_PAUSEPROB=0.99 # what is this value?
        loop_prefactor=1.5 # what is this value?
        FULL_LOOP_ENTROPY=1 # what is this value?
        FRACTION_ONESIDED=0

        # Extruder dynamics frequency of sampling parameters #
        numExtruderSteps = 1000 # steps taken for each simulation sample
        numInitializationSteps = 10000 # how long we take to equilibrate the simulation


        # Polymer and extruder dynamics parameters #
        L = 10000 
        processivity = lof[idx][3]#100
        separations = lof[idx][4]#2*processivity
#         TAD_size = 200
#         TAD_boundaries = np.arange(0,L,TAD_size)
        PAUSEPROB=0.0 # motor pause probability


        smcNum = L//separations # number of SMCs loaded  
        SWITCH =  np.ones(L,dtype=np.double)*SWITCH_PROB
        LIFETIME = processivity
        birthArray = np.ones(L)/L
        deathArray = np.zeros(L, dtype=np.double) + 1. / LIFETIME
        deathArray[0:1] = 1
        deathArray[L-2:L-1] = 1
        stallDeathArray = deathArray 
        stallDeathArray[0:1] = 1
        stallDeathArray[L-2:L-1] = 1    
        pauseArray = np.zeros(L, dtype=np.double) + PAUSEPROB 
        slidePauseArray = np.zeros(L, dtype=np.double) + SLIDE_PAUSEPROB
        oneSidedArray = np.zeros(smcNum, dtype=np.int64)
        belt_on_array = np.zeros(smcNum, dtype=np.double) + BELT_ON
        belt_off_array = np.zeros(smcNum, dtype=np.double) + BELT_OFF
        spf=slidePauseArray*(1.-(1.-SLIDE_PAUSEPROB)*np.exp(-1.*loop_prefactor))
        spb=slidePauseArray*(1.-(1.-SLIDE_PAUSEPROB)*np.exp(loop_prefactor))
        ################### TAD BOUNDARY CONDITION###################
        stallLeftArray = np.zeros(L, dtype = np.double)
        stallRightArray = np.zeros(L,dtype = np.double)
        stallprob = 0.4
        for b in TAD_boundaries:
            stallLeftArray[b] = stallprob
            stallRightArray[b] = stallprob
        ##################################################################
        ################### Random barrier CONDITION###################
        numRandomBarriers = int(np.round(len(TAD_boundaries)*lof[idx][0]))
        randomBarriers = np.random.choice(L,numRandomBarriers,replace=False)
        boundaryPauseProb = lof[idx][1]
        for b in randomBarriers:
            stallLeftArray[b] = boundaryPauseProb
            stallRightArray[b] = boundaryPauseProb

        ##################################################################    


        transloc = smcTranslocatorDirectional(birthArray, deathArray, stallLeftArray, stallRightArray, pauseArray,
                                             stallDeathArray, smcNum, oneSidedArray, FRACTION_ONESIDED, slide=SLIDE,
                                             slidepauseForward=spf, slidepauseBackward=spb, switch=SWITCH, pushing=PUSH,
                                            belt_on=belt_on_array, belt_off=belt_off_array,SLIDE_PAUSEPROB=SLIDE_PAUSEPROB) 

        transloc.steps(numInitializationSteps)


        ### dump empty data as test
        X = []
        Y = []
        P = []
        C = []
        loopSizes_array = []
        L_tot = L
        Z = coo_matrix((P,(X,Y)),shape=(L_tot,L_tot))
        Zcount = coo_matrix((C,(X,Y)),shape=(L_tot,L_tot))
        ChIP = np.zeros(L)

        param_sweep_dict = {'ChIP':ChIP,'Z':Z,'Zcount':Zcount,\
                    'RandomBarriers':randomBarriers,\
                   'nSMCs':smcNum,\
                   'numReps':nreps,'nsamples_per_map':nsamples,'lifetime':LIFETIME,\
                        'TAD_boundaries':TAD_boundaries,\
                       'randomBoundaryPauseProb':boundaryPauseProb,'TADBoundaryPauseProb':stallprob,'loopSizesArray':loopSizes_array}


        pickle.dump(param_sweep_dict,open(filename,'wb'))

        print("Doing {}".format(filename))



        for ic in range(nreps):
            transloc.steps(numExtruderSteps)
            # generate chain using loop and gap statistics
            smcs_lr = transloc.getSMCs()
            
            # get ChIP-seq
            for x in range(len(smcs_lr[0])):                
                ChIP[smcs_lr[0][x]] += 1
                ChIP[smcs_lr[1][x]] += 1
            
            
            smcs = [(smcs_lr[0][x], smcs_lr[1][x]) for x in range(len(smcs_lr[0]) )  ]
            cl = None
            try:
                if len(smcs)==0:
                    smcs = [(0,1)]
                cl = ChainLayout(smcs,L_tot)
            except:
                print("Pseudoknot formed count:{}".format(ic))
                print(cl)
                assert(1==0)
                cl = None
            if cl is None:
                continue
                
            # measure loop sizes
            loopSizes_array = loopSizes_array + list(smcs_lr[1]-smcs_lr[0])
            
            # get contact maps, subsampling from distribution 
            vals = sorted(np.random.choice(L_tot,nsamples, replace=False))
            for ix in range(len(vals)):
                for iy in range(ix,len(vals)):
                    x = vals[ix]
                    y = vals[iy]
                    deff = cl.get_dist(x,y)
                    if deff == 0:
                        pc = 1
                    else:
                        pc = 1/np.sqrt(deff)**3
                    if not np.isnan(pc):
                        X.append(x)
                        Y.append(y) # x
                        P.append(pc) # probability
                        C.append(1) # counts

        Z = coo_matrix((P,(X,Y)),shape=(L_tot,L_tot))
        Zcount = coo_matrix((C,(X,Y)),shape=(L_tot,L_tot))


        param_sweep_dict = {'ChIP':ChIP,'Z':Z,'Zcount':Zcount,\
                    'RandomBarriers':randomBarriers,\
                   'nSMCs':smcNum,\
                   'numReps':nreps,'nsamples_per_map':nsamples,'lifetime':LIFETIME,\
                        'TAD_boundaries':TAD_boundaries,\
                       'randomBoundaryPauseProb':boundaryPauseProb,'TADBoundaryPauseProb':stallprob,'loopSizesArray':loopSizes_array}

        pickle.dump(param_sweep_dict,open(filename,'wb'))
        
        return 1
    except:
        return 0
#     return 1

pool = Pool(len(lof))
pool.map(doOne, np.arange(len(lof)))
# pool = Pool(30)
# pool.map(doOne, np.arange(len(lof)))

Doing ./RandomBoundarySims_refined/FixedTADs_proc60_sep200_tadsize300_fractionBoundaries0_boundaryPauseProb0_nsamples1000_nreps70.pkl
Doing ./RandomBoundarySims_refined/FixedTADs_proc75_sep200_tadsize300_fractionBoundaries0_boundaryPauseProb0_nsamples1000_nreps70.pkl
Doing ./RandomBoundarySims_refined/FixedTADs_proc95_sep200_tadsize300_fractionBoundaries0_boundaryPauseProb0_nsamples1000_nreps70.pkl
Doing ./RandomBoundarySims_refined/FixedTADs_proc100_sep200_tadsize300_fractionBoundaries0_boundaryPauseProb0_nsamples1000_nreps70.pkl
Doing ./RandomBoundarySims_refined/FixedTADs_proc60_sep150_tadsize300_fractionBoundaries0_boundaryPauseProb0_nsamples1000_nreps70.pkl
Doing ./RandomBoundarySims_refined/FixedTADs_proc75_sep150_tadsize300_fractionBoundaries0_boundaryPauseProb0_nsamples1000_nreps70.pkl
Doing ./RandomBoundarySims_refined/FixedTADs_proc100_sep150_tadsize300_fractionBoundaries0_boundaryPauseProb0_nsamples1000_nreps70.pkl
Doing ./RandomBoundarySims_refined/FixedTADs_proc95_sep150_t